# Purpose of this notebook

Do something useful with the  [relational data model](https://opendata.tweedekamer.nl/documentatie/informatiemodel) that the tweede kamer provides -- see also 
[extras_datacollect_tweede_kamer_dossiers](extras_datacollect_tweede_kamer_dossiers.ipynb).


Our quest is "make a list of Who is part of what party.

This involves four different resources: Persoon, FractieZetelPersoon, FractieZetel, Fractie

You can see FractieZetelPersoon is a many-to-many joiner table
- a reference to a FractieZetel, which itself just references a Fractie
- a reference to a and Persoon

Persoon and Fractie have no references so are just details to each item.

The below code does relational joins for party membership, 
then flattens the results by adding membership to the persoon details.

In [1]:
import json
import pprint

import wetsuite.datacollect.tweedekamer_nl   # contains some basic code dealing with this API
import wetsuite.helpers.etree

In [2]:
# Fetch all resources of the mentioned soort.
# We need four of the. The combination should take a minute or two.
print("Fetching Persoon")
data_Persoon            = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'Persoon' ) )
print("Fetching Fractie")
data_Fractie            = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'Fractie' ) )
print("Fetching FractieZetel")
data_FractieZetel       = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'FractieZetel' ) )
print("Fetching FractieZetelPersoon")
data_FractieZetelPesoon = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'FractieZetelPersoon' ) )

Fetching Persoon
Fetching Fractie
Fetching FractieZetel
Fetching FractieZetelPersoon


In [14]:
print( wetsuite.helpers.etree.debug_pretty(data_FractieZetelPesoon.find('entry')) )

<entry>
  <title>808fcd50-a0dc-4f60-8b9d-c404a2eb5b2e</title>
  <id>https://gegevensmagazijn.tweedekamer.nl/SyncFeed/2.0/Entiteiten/808fcd50-a0dc-4f60-8b9d-c404a2eb5b2e</id>
  <author>
    <name>Tweede Kamer der Staten-Generaal</name>
  </author>
  <updated>2023-08-29T13:23:13Z</updated>
  <category term="fractieZetelPersoon"/>
  <link rel="next" href="https://gegevensmagazijn.tweedekamer.nl/SyncFeed/2.0/Feed?category=FractieZetelPersoon&amp;skiptoken=16687327"/>
  <content type="application/xml">
    <fractieZetelPersoon id="808fcd50-a0dc-4f60-8b9d-c404a2eb5b2e" bijgewerkt="2023-08-29T11:10:24Z" verwijderd="false">
      <fractieZetel ref="ca826e72-cf57-4cca-b090-d5c444ec6c2d"/>
      <persoon ref="ec273841-069f-408b-b434-8524904ae314"/>
      <functie>Lid</functie>
      <van>2002-05-23</van>
      <totEnMet>2010-10-11</totEnMet>
    </fractieZetelPersoon>
  </content>
</entry>



In [ ]:
wetsuite.datacollect.tweedekamer_nl.entry_dicts( data_FractieZetelPesoon )

In [46]:
# Fetched this way, we need to do what amounts to a manual JOIN of relational data

# We can make life simpler for ourselfes by making it easier to fetch varied individual items:
#   split into items, let us look up by id, and by type
id_thing = {}   #               guid -> detailsdict
by_type  = {}   #     soort/category -> list of all such detailsdicts

for etree in (data_Persoon, data_Fractie, data_FractieZetel, data_FractieZetelPesoon):
    for entry_dict in wetsuite.datacollect.tweedekamer_nl.entry_dicts( etree ):
        id_thing[ entry_dict['content']['id'] ] = entry_dict

        category = entry_dict['category']
        if category not in by_type:
            by_type[category] = []
        by_type[ category ].append( entry_dict )

In [52]:
# The join-y code
persoon_combined = {}  # which is mostly still the persoon dicts, except we added a key that is the membership

for fzp_dict in by_type['fractieZetelPersoon']:
    content = fzp_dict['content']

    if 'fractieZetel' not in content['refs']:
        print("  FractieZetelPersoon item without FractieZetel - huh? %r"%fzp_dict)
        continue

    fractiezetel_id = content['refs']['fractieZetel']
    persoon_id      = content['refs']['persoon']

    fractiezetel_dict = id_thing[ fractiezetel_id ]
    persoon_dict      = id_thing[ persoon_id ]

    fractie_id = fractiezetel_dict['content']['refs']['fractie']
    frac_dict = id_thing[ fractie_id ]

    if persoon_id not in persoon_combined:
        persoon_combined[ persoon_id ] = persoon_dict
        persoon_combined[ persoon_id ]['fractie_membership'] = []
        
    persoon_combined[ persoon_id ]['fractie_membership'].append( 
        {
            'fractie_id':frac_dict['content']['id'], 
            'fractie_afkorting':frac_dict['content']['afkorting'], 
            # maybe more or all of frac_dict?
            'functie':fzp_dict['content']['functie'], 
            'van':fzp_dict['content']['van'], 
            'totEnMet':fzp_dict['content']['totEnMet'],
        } 
    )

  FractieZetelPersoon item without FractieZetel - huh? {'title': 'd73d7f69-1235-4746-aa94-84b593909bfc', 'updated': '2023-08-29T14:00:09Z', 'category': 'fractieZetelPersoon', 'content': {'refs': {}, 'tagname': 'fractieZetelPersoon', 'id': 'd73d7f69-1235-4746-aa94-84b593909bfc', 'bijgewerkt': '2023-08-29T11:10:32Z', 'verwijderd': 'true'}, 'id': 'https://gegevensmagazijn.tweedekamer.nl/SyncFeed/2.0/Entiteiten/d73d7f69-1235-4746-aa94-84b593909bfc'}
  FractieZetelPersoon item without FractieZetel - huh? {'title': '8cca26af-365a-46fc-b72f-c42b2a17a992', 'updated': '2023-12-06T13:48:37Z', 'category': 'fractieZetelPersoon', 'content': {'refs': {}, 'tagname': 'fractieZetelPersoon', 'id': '8cca26af-365a-46fc-b72f-c42b2a17a992', 'bijgewerkt': '2023-12-05T14:58:31Z', 'verwijderd': 'true'}, 'id': 'https://gegevensmagazijn.tweedekamer.nl/SyncFeed/2.0/Entiteiten/8cca26af-365a-46fc-b72f-c42b2a17a992'}
  FractieZetelPersoon item without FractieZetel - huh? {'title': '6d685347-a5c8-498b-94df-b7bbb1bc12

In [53]:
fracs = {} # id -> details, for consistency with the above and easier lookup
for fracs_dict in by_type['fractie']:
    fracs[ fracs_dict['id'] ] = fracs_dict

In [78]:

print("WRITING tweedekamer-fractie-membership-struc.json")
with open('tweedekamer-fractie-membership-struc.json','wb') as jsonfile:
    jsonfile.write( json.dumps({ 
        'description_short':''' Description of people, including party memberships over time. ''',
        'description':      ''' Description of people, including party memberships over time(each with fractie_afkorting, functie, van, totEnMet) ''',
        'data':persoon_combined} ).encode('ascii')
    )

print("WRITING tweedekamer-fracties-struc.json")
with open('tweedekamer-fracties-struc.json','wb') as jsonfile:
    jsonfile.write( json.dumps({ 
        'description_short':''' Description of political parties/fracties. ''',
        'description':      ''' Description of political parties/fracties.

    Items look something like: 
        {'title': 'ae48391e-ce4d-47e0-86e3-ee310282f66f',
        'updated': '2023-12-06T13:48:37Z',
        'category': 'fractie',
        'nummer': '50311',
        'afkorting': 'Volt',
        'naamNl': 'Volt',
        'naamEn': 'Volt',
        'aantalZetels': '2',
        'aantalStemmen': '178802',
        'datumActief': '2021-03-31',
        'datumInactief': None,
        'id': 'ae48391e-ce4d-47e0-86e3-ee310282f66f'}
        ''',
        'data':fracs} ).encode('ascii')
    )


WRITING tweedekamer-fractie-membership-struc.json
WRITING tweedekamer-fracties-struc.json
